In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import requests

In [43]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
columns = ["Country", 'Continent', "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

In [65]:
def extract(url, columns):
    page_text = requests.get(url).text
    soup = BeautifulSoup(page_text, 'html.parser')
    df = pd.DataFrame(columns=columns)

    table = soup.find('table', class_='wikitable')
    rows = table.find_all('tr')

    for row in rows:
        tds = row.find_all('td')
        if len(tds) > 0 and tds[0].find('a') and tds[2].text.strip() != '—':
          extracted_data = pd.DataFrame({
              'Country': [tds[0].find('a').text],
              'Continent': [tds[1].find('a').text],
              'GDP_USD_millions': [tds[2].text]
          }, index=[0])
          df = pd.concat([df,extracted_data], ignore_index=True)
    return df

In [66]:
data = extract(url, columns)
print(data.head())

         Country Continent GDP_USD_millions
0  United States  Americas       26,854,599
1          China      Asia       19,373,586
2          Japan      Asia        4,409,738
3        Germany    Europe        4,308,854
4          India      Asia        3,736,882


In [67]:
def transform(df):
  df['GDP_USD_millions'] = df['GDP_USD_millions'].str.replace(',', '')
  df['GDP_USD_millions'] = df['GDP_USD_millions'].astype(int)
  df['GDP_USD_millions'] /= 1000
  df = df.rename(columns={'GDP_USD_millions':'GDP_USD_billions'})
  return df

In [68]:
transformed_data = transform(data)

In [69]:
def load_to_csv(df, path):
  df.to_csv(path)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [70]:
sql_connection = sqlite3.connect('World_Economies.db')

In [71]:
load_to_csv(transformed_data, csv_path)
load_to_db(transformed_data, sql_connection, table_name)

In [74]:
print(pd.read_sql(f"SELECT COUNT(*) from {table_name}", sql_connection))

   COUNT(*)
0       191


In [73]:
print(pd.read_sql(f"SELECT Country, GDP_USD_billions from {table_name} WHERE GDP_USD_billions >= 100", sql_connection))

          Country  GDP_USD_billions
0   United States         26854.599
1           China         19373.586
2           Japan          4409.738
3         Germany          4308.854
4           India          3736.882
..            ...               ...
64          Kenya           118.130
65         Angola           117.877
66           Oman           104.902
67      Guatemala           102.309
68       Bulgaria           100.635

[69 rows x 2 columns]


In [75]:
print(pd.read_sql(f"SELECT Continent, AVG(GDP_USD_billions) as Average_GDP from {table_name} GROUP BY Continent ORDER BY Average_GDP DESC", sql_connection))

  Continent  Average_GDP
0  Americas   982.939028
1      Asia   905.602886
2    Europe   578.796791
3   Oceania   142.995786
4    Africa    55.450778
